In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import json
import folium # Check description below if not already installed

from datetime import date

data_foleder = 'data'
json_folder = 'topojson'

unemployement_eu = os.path.join(data_foleder, 'eurostat.csv')
unemployement_ch = os.path.join(data_foleder, 'unemployement_rate_ch.xlsx')
unemployement_sex_ch = os.path.join(data_foleder, 'unemployement_rate_sex_ch.xlsx')
unemployement_factor_ch = os.path.join(data_foleder, 'unemployement_rate_factor_ch.xlsx')
unemployement_nat_ch = os.path.join(data_foleder, 'unemployement_rate_nat_ch.xlsx')
topojson_ch = os.path.join(json_folder, 'ch-cantons.topojson.json')
topojson_eu = os.path.join(json_folder, 'europe.topojson.json')

eu_infos = {'lat_long': [54.5, 15.3], 'zoom': 3, 'key_on': 'NAME', 'topojson': 'objects.europe'}
ch_infos = {'lat_long': [46.8, 8.2], 'zoom': 8, 'key_on': 'name', 'topojson': 'objects.cantons'}

# 0. Description 

## Background
In this homework we will be exploring interactive visualization, which is a key ingredient of many successful data visualizations (especially when it comes to infographics).

Unemployment rates are major economic metrics and a matter of concern for governments around the world. Though its definition may seem straightforward at first glance (usually defined as the number of unemployed people divided by the active population), it can be tricky to define consistently. For example, one must define what exactly unemployed means : looking for a job ? Having declared their unemployment ? Currently without a job ? Should students or recent graduates be included ? We could also wonder what the active population is : everyone in an age category (e.g. `16-64`) ? Anyone interested by finding a job ? Though these questions may seem subtle, they can have a large impact on the interpretation of the results : `3%` unemployment doesn't mean much if we don't know who is included in this percentage. 

In this homework you will be dealing with two different datasets from the statistics offices of the European commission ([eurostat](http://ec.europa.eu/eurostat/data/database)) and the Swiss Confederation ([amstat](https://www.amstat.ch)). They provide a variety of datasets with plenty of information on many different statistics and demographics at their respective scales. Unfortunately, as is often the case is data analysis, these websites are not always straightforward to navigate. They may include a lot of obscure categories, not always be translated into your native language, have strange link structures, â€¦ Navigating this complexity is part of a data scientists' job : you will have to use a few tricks to get the right data for this homework.

For the visualization part, install [Folium](https://github.com/python-visualization/folium) (*HINT*: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!). Folium's `README` comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find two `.topojson` files, containing the geo-coordinates of 

- European countries (*liberal definition of EU*) (`topojson/europe.topojson.json`, [source](https://github.com/leakyMirror/map-of-europe))
- Swiss cantons (`topojson/ch-cantons.topojson.json`) 

These will be used as an overlay on the Folium maps.

## Assignment

1. Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

2. Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

3. Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

4. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [RÃ¶stigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?

## Folium

#### Installation

We recommend using the latest version : `0.5.0`.

`Folium` is a regular python package, which can be installed through several means :

#### 1. Conda
```
conda install folium
```

#### 2. pip

```
pip install -U folium
``` 

By default, the `pip` command is linked to the local `python` distribution on. To use with your notebook, make sure you use the pip bundled with `anaconda`. On mac for example, this is usually : 
```
~/anaconda/bin/pip install -U folium
```

#### 3. Direct download

The package is available [directly from pypi](https://pypi.python.org/pypi/folium)

---

In [ ]:
def plot_choropletch(df, col_val, json_data, legend='', info=ch_infos, colors=['green','yellow','red']):
    # Create map and colormap
    _map = folium.Map(location = info['lat_long'], zoom_start = info['zoom'], tiles='cartodbpositron')
    linear = folium.LinearColormap(colors, vmin=df[col_val[1]].min(), vmax=df[col_val[1]].max())
    # Define style function that return color as a function of the input value
    def style_color(feature):
        id_geo = df.reset_index()[col_val[0]] == feature['properties'][info['key_on']]
        return {'fillColor': linear(
                df.reset_index().loc[id_geo, col_val[1]].values ),
                'color' : 'black', 'weight' : 1, 'dashArray' : '5, 5', 'opacity': 1}   
    # Add color overlay based on json data
    folium.TopoJson(json_data,
                    object_path = info['topojson'],
                    style_function = style_color).add_to(_map)
    # Add legend and caption
    linear.caption = legend
    _map.add_child(linear)
    return _map

# 1. Eurostat

We can find on the Eurostat website many interesting european statistics. For our project we are going to use this specific one: [Employment rates by sex, age and citizenship](http://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=lfsq_ergan&lang=en). It gathers the unemployment rates of every european countries from 2015 first quarter to 2017 second quarter.
We choose to download it into the .csv format and then import it into a Panda Dataframe for more data handling.

In [ ]:
eurostat = pd.read_csv(unemployement_eu)
eurostat.head(5)

We can now look at the range of each fields.
* `TIME` is expressed as quarter (year split in 4)
* `SEX` is not a binary field and is composed of Males, Females and Total fields.
* `AGE` is a single range : From 15 to 64 years, so we would discard it (not relevant)
* `CITIZEN` is always equals to "Total", so not relevant and will be discarded
* `UNIT` is just the unit of `Value` column
* `Value` is the emplyement rate for each country. Goes from 28.9 to 91.4 (huge gap!).
* `Flag and Footbotes`: is also an non-relevant field and will be discarded

In [ ]:
print('Values in TIME: {}'.format(eurostat['TIME'].unique()))
print('Values in SEX: {}'.format(eurostat['SEX'].unique()))
print('Values in AGE: {}'.format(eurostat['AGE'].unique()))
print('Values in CITIZEN: {}'.format(eurostat['CITIZEN'].unique()))
print('Values in UNIT: {}'.format(eurostat['UNIT'].unique()))
print('Values in Value: {} to {}'.format(eurostat['Value'].min(), eurostat['Value'].max()))
print('Values in Flag and Footnotes: {}'.format(eurostat['Flag and Footnotes'].unique()))

* `GEO` is composed of the names of the countries and some overall stats of EU. We can discard the general values since we are only interested in country-wise values. We also note that Gernamy has a really formal name as "Germany (until 1990 former territory of the FRG)"

In [ ]:
print('Values in GEO: {}'.format(eurostat['GEO'].unique()))

We can convert `TIME` to actual timestamp. For example 2016Q2 ends in 2016 + 6 months aka, June 2016. Aso we remove non-relevant fields such as `AGE`, `CITIZEN`, `UNIT`, `Flag and Footnotes`. The we take only the lastest entries, i.e 2017Q2.

In [ ]:
def convert_time(time):
    quarter = int(time[time.find('Q')+1])
    year = int(time[0:4])
    return pd.Timestamp(date(year, quarter*3, 1) ) 

eurostat.drop(['AGE', 'CITIZEN', 'UNIT', 'Flag and Footnotes'], axis=1, inplace=True)
eurostat.TIME = eurostat.TIME.apply(convert_time)
eurostat = eurostat.loc[eurostat.TIME == eurostat.TIME.max()]

As expected 2017Q2 is the latest entry and correpond to June 2017.

In [ ]:
eurostat.TIME.value_counts()

We remove as well European Total fiels fields that we do not need for plotting.

In [ ]:
drop_geo = ['European Union (28 countries)', 'European Union (27 countries)',
            'European Union (15 countries)', 'Euro area (19 countries)', 
            'Euro area (18 countries)', 'Euro area (17 countries)']

eurostat = eurostat.loc[[name not in drop_geo for name in eurostat.GEO]]

Now that our dataframe is clean we need to be sur that country as correctly matched with TOPOJSON data. To do so We will print country for both (as a visual purpose) and then look for missig matches.

We can notice that we do not have data for all countries. For example we have no data for Azerbaijan which makes actually sense since Azerbaijan is not part of EU (according to political point of view). 

In [ ]:
print('Countries Eurostat:\n{}'.format(eurostat.GEO.unique()))

In [ ]:
data_topojson_eu = json.load(open(topojson_eu))
geo_eu_countries = []
for country in data_topojson_eu['objects']['europe']['geometries']:
    geo_eu_countries.append(country['properties']['NAME'])
print('Countries JSON EU:\n{}'.format(geo_eu_countries))

We can see that two country in our data are not located in JSON file. `Germany` is `Germany (until 1990 former territory of the FRG)` and `The former Yugoslav Republic of Macedonia` is `Former Yugoslav Republic of Macedonia, the` in eurostat data.

In [ ]:
eurostat.loc[ [country not in geo_eu_countries for country in eurostat.reset_index().GEO], 'GEO' ].unique()

We can directly replace thoses countries name in our dataset so both data will match.

In [ ]:
eurostat.GEO.replace(
    {'Germany (until 1990 former territory of the FRG)': 'Germany',
     'Former Yugoslav Republic of Macedonia, the': 'The former Yugoslav Republic of Macedonia'}, inplace=True)

Note that we need as well to discard countries in our TOPOJSON to avoid confusion in ploting results. We will therefore only keep countries that are matching in **both** data (eurostat and topojson). 

In [ ]:
json_id_keep = [country['properties']['NAME'] in eurostat.GEO.values 
                for country in data_topojson_eu['objects']['europe']['geometries']]
data_topojson_eu['objects']['europe']['geometries'] = np.array(data_topojson_eu['objects']['europe']['geometries'])[json_id_keep].tolist()

We can reorganize our dataframe so Males, Females and Total fileds will be presented as unique columns. This will allow us to easily compute differences in quantities. Note that we also change our data to unemployment as $\text{uemployement} = 100-\text{employement}$ if both fiels are expressed as a percentage.

In [ ]:
eurostat = eurostat.pivot_table(index=['GEO', 'TIME'], columns='SEX', values='Value')
eurostat = 100-eurostat
eurostat['Females_o_Males'] = eurostat['Females']/eurostat['Males']
eurostat.head()

## 1.1 Results

If we plot now the results we can observe that Nordic countries have, in avergage, a lower unemployment rate. South europe shows the higher unemployement rate. Not that Finland acts like an outlier since it should be part of Nordic countries. Note that according to [Tradingeconomics Finland Unemployement](https://tradingeconomics.com/finland/unemployment-rate) the unemployement is 8.9% for June 2017. Apparentelly there are huge difference in the way they compute unemployement rate.

If we look more carefully at thoses data, we can observe that unemployement in EU is computed as `not_working_population`/`active_population` ([Definition here](http://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Employment_rate)) which will of course gives huge unemployement values. This is indeed the literal interpetation of the term "unemployemnet" but not representative at all of the actual country situation.

In [ ]:
europe_map = plot_choropletch(eurostat, ['GEO', 'Total'], data_topojson_eu, 
                                 legend='EU unemployement per country in %', info=eu_infos)
europe_map

It is funny to see that South-Eastern countries have higher unemployement rate for women. Which is porbabliy linked to cultural reasons since woman are less likely to work and more likely to be housewives.

In [ ]:
europe_map = plot_choropletch(eurostat, ['GEO', 'Females_o_Males'], data_topojson_eu, 
                              legend='EU ratio difference in Female and Males unemployement', info=eu_infos)
europe_map

We can keep in mind those values to compare them with Switzerland offical ones.

In [ ]:
print('European unemployment values for Swizzerland:\n\n{}'.format(
    eurostat.loc[('Switzerland', '2017-06-01'), ['Total', 'Males', 'Females']]))

---
# 2. Switzerland

To compare results with EU ones, we chosed to also load data for unemployement for both gender. Our data were taken form [amsat](https://www.amstat.ch/v2/index.jsp?lang=fr) as asked.

Both data contain common fields `canton` that will allows us to merge them easily. Note that data gender names are in German. We replace the with english equivalent to make display more coherent.

In [ ]:
ch_stat = pd.read_excel(unemployement_ch, skiprows=4, header=None, usecols=np.array([0,2]))
ch_stat_sex = pd.read_excel(unemployement_sex_ch, skiprows=4, header=None, usecols=np.array([0,1,3]))

ch_stat.columns = ['canton', 'rate']
ch_stat_sex.columns = ['canton', 'sex', 'rate']

ch_stat = pd.concat((ch_stat, ch_stat_sex))
ch_stat.replace({np.NAN: 'Total', 'Männer': 'Males', 'Frauen': 'Females'}, inplace=True)
ch_stat.sort_values(by='canton').head(6)

Here we are facing the same problem as EU JSON data. We need to make sure each canton are correctly linked. We can take a look at both canton name in TOPOJSON file and in Amstat data.

In [ ]:
data_topojson_ch = json.load(open(topojson_ch))
geo_ch_cantons = []

for canton in data_topojson_ch['objects']['cantons']['geometries']:
    geo_ch_cantons.append(canton['properties']['name'])
print('Countries JSON Switzerland:\n{}'.format(geo_ch_cantons))

In [ ]:
print('Countries Amstat Switzerland :\n{}'.format(ch_stat.canton.unique()))

Now let's look at canton that are **not** correctly matched together

In [ ]:
ch_stat.loc[ [canton not in geo_ch_cantons for canton in ch_stat.canton], 'canton' ].unique()

We can see that canton in TopoJSON are named according to their local language (e.i. Vaud is french and Basel-Stadt is German). In Amstat data all canton are in German. It is therfore not possible to quickly match not Gemran canton together. We used therefore a dictionnary to match them. We create a new column named `canton_json` so we keep track of original names.

Note that one line is 'Gesamt' which mean 'Total' in German We can discrad that row as well.

In [ ]:
ch_stat['canton_json'] = ch_stat.canton
ch_stat['canton_json'].replace(
                    {'Bern': 'Bern/Berne', 'Freiburg': 'Fribourg', 'Graubünden': 'Graubünden/Grigioni', 
                     'Tessin': 'Ticino', 'Waadt': 'Vaud', 'Wallis': 'Valais/Wallis',
                     'Neuenburg': 'Neuchâtel', 'Genf': 'Genève'}, inplace=True)
ch_stat = ch_stat[ch_stat.canton != 'Gesamt']
ch_stat.head()

We can do the same as we did for EU. We will take total, males and females as distinct columns and compute the ratio of unemployement between the two genters.

In [ ]:
ch_stat = ch_stat.pivot_table(index=['canton', 'canton_json'], columns='sex', values='rate')
ch_stat.columns = ['females_rate', 'males_rate', 'total_rate']
ch_stat['female_o_males'] = ch_stat['females_rate']/ch_stat['males_rate']
ch_stat.reset_index(inplace=True)
ch_stat.head()

## 2.1 Results - General
We can see a trend in our data. The french seaking part of Switzerland (West) have higher unemployement rates. Also inner switzerland (Middle/East) have in average a low level of unemployement.

In [ ]:
ch_map = plot_choropletch(ch_stat, ['canton_json', 'total_rate'], json_data= data_topojson_ch,
                          legend='Swiss unemployement percentage for Total population')
ch_map

We can see that in average both `Males` and `Females` have the same level of unemployement. Except maybe for Uri where the ratio goes upto 1.4.

In [ ]:
ch_map = plot_choropletch(ch_stat, ['canton_json', 'female_o_males'], json_data= data_topojson_ch,
                          legend='Swiss ratio difference in Female and Males unemployement')
ch_map

## 2.2 Results - Difference in interpretation

So we just saw that there is a huge difference in unemployment between the values given by EU (~20%) and Switzerland (Not calculated yet but at most 5.2% according to Geneve canton situation). To understant why their is such a huge difference let's decribe how unemployement is ccomputed in Switzerland ([Source](https://www.seco.admin.ch/seco/fr/home/wirtschaftslage---wirtschaftspolitik/Wirtschaftslage/Arbeitslosenzahlen.html)).

1. You are considered as a `job seeker` if you are registred to an ORP (Office régional de placement).
2. You are considered as `unemployed` if you are subscribed to an ORP **AND** if you recieve subsidies from the canton. 
3. You are a `job seeker` but **not** `unemployed` if you are registered to an ORP but have no rights to subsidies (e.g. You haven't worked enough or are registred to social aid).
4. Switzerland is computing unemploymenet as `unemployed`/`active_population`.

Here we will add to our dataframe the actual number of `Job Seekers`, `Unemplyoed`, `Unemployed (Young)`, `Long Term Unemployed` (unemployed for > 1 year).

In [ ]:
ch_stat_fact = pd.read_excel(unemployement_factor_ch, skiprows=4, header=None, 
                             usecols=np.array([0,3,4,5,6,7]))
ch_stat_fact.columns = ['canton', 'unemp_number','unemp_young', 
                   'unemp_longterm', 'unemp_seeker_number', 'seeker']
ch_stat_fact = ch_stat_fact[ch_stat_fact.canton != 'Gesamt']
ch_stat_fact.head()

Since we have the same names for the canton, we can merge those data to our dataframe

In [ ]:
ch_stat = ch_stat.merge(ch_stat_fact)
ch_stat.head()

To be able to compute the new rates we need to know the actual activae population. Therefore according to the definition above `Active Population` = (100/`Total Rate`) * `Number of unemployed`. Now that we have the amount of active population we can estimate:

* `unemp_rate_seeker` Unemployement rate considering all people that are looking for a job, e.i. all people registred to an ORP.
* `unemp_rate_no_young` Unemployement rate but without considering young people (under 25).
* `unemp_rate_longterm` Unemployement rate considering only people that are unemployed for more than a year.

For display purpose we will plot the difference between thoses new fields and the official unemployement value. This will give us an estimate of the importance of each factor per canton.

In [ ]:
# Active population
ch_stat['canton_active_pop'] =  (100/ch_stat.total_rate) * ch_stat.unemp_number
# New ratio as explained
ch_stat['unemp_rate_seeker'] = 100*(ch_stat.unemp_seeker_number/ch_stat['canton_active_pop'])
ch_stat['unemp_rate_no_young'] = 100*((ch_stat.unemp_number-ch_stat.unemp_young)/ch_stat['canton_active_pop'])
ch_stat['unemp_rate_longterm'] = 100*((ch_stat.unemp_longterm)/ch_stat['canton_active_pop'])
# Difference in between ratio and official data of unemployement (for vizualization purpose)
ch_stat['unemp_rate_seeker_diff'] = ch_stat.unemp_rate_seeker - ch_stat.total_rate
ch_stat['unemp_rate_no_young_diff'] = ch_stat.unemp_rate_no_young - ch_stat.total_rate
ch_stat['unemp_rate_longterm_diff'] = ch_stat.unemp_rate_longterm - ch_stat.total_rate
ch_stat.head()

### 2.2.1 Job seekers

If we take all job seekers we will get higher values for canton that have an larger amount of foreigner workers. This might be due to the fact that some canton such as Jura, Ticino or Schaffusen have a larger amount of personne coming from EU to work. This will results in an higher percentage of job seeker in those area.

In [ ]:
ch_map = plot_choropletch(ch_stat, ['canton_json', 'unemp_rate_seeker_diff'], json_data= data_topojson_ch,
                          legend='Percentage unemployement for Total poulation')
ch_map

### 2.2.2 Without young people

Here we see that Neuchatel and Jura have the highest amount of unemployement for young people. This might be link to the fact that this region of Switzerland is directly linked to watchmaking which is facing an economic crisis.

In [ ]:
ch_map = plot_choropletch(ch_stat, ['canton_json', 'unemp_rate_no_young_diff'], json_data= data_topojson_ch,
                          legend='Percentage unemployement for Total poulation', colors=['red', 'yellow', 'green'])
ch_map

### 2.2.3 Long term Unemployement

Same logic here. Since more people recently lost their job linked to the economic crisis (watchmaking sector) they are less likely to find an new job in this area and therefore more likely to stay longer in an unemplyoement situation.


In [ ]:
ch_map = plot_choropletch(ch_stat, ['canton_json', 'unemp_rate_longterm'], json_data= data_topojson_ch,
                          legend='Percentage unemployement for Total poulation')
ch_map

### 2.2.4 Overall

We can display our results for the whole country for thoses different type of factors. As we can see we indeed get the same unemployment rate (3%) as the one published on [Amstat](https://www.amstat.ch/v2/index.jsp?lang=fr) website. We can see that this huge difference with EU values (12%) is only due to different ways of computing the same value.

In [ ]:
swiss_unemp = 100*ch_stat.unemp_number.sum()/ch_stat.canton_active_pop.sum()
swiss_unemp_seek = 100*ch_stat.unemp_seeker_number.sum()/ch_stat.canton_active_pop.sum()
swiss_unemp_no_young = 100*(ch_stat.unemp_number.sum()-ch_stat.unemp_young.sum())/ch_stat.canton_active_pop.sum()
swiss_unemp_longtem = 100*ch_stat.unemp_longterm.sum()/ch_stat.canton_active_pop.sum()

print('{:.2}% : Unempl. rate'.format(swiss_unemp))
print('{:.2}% : Unempl. rate with Job Seeker'.format(swiss_unemp_seek))
print('{:.2}% : Unempl. rate without Young people'.format(swiss_unemp_no_young))
print('{:.2}% : Unempl. rate Long Term'.format(swiss_unemp_longtem))

## 2.3 Foreigners and Age classes
